In [ ]:
import numpy
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import RobertaForSequenceClassification
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [ ]:
#carregando o dataset go_emotions do HuggingFace
dataset = load_dataset('go_emotions')

In [ ]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [ ]:
dataset['train'][0], len(dataset['train'])

({'text': "My favourite food is anything I didn't have to cook myself.",
  'labels': [27],
  'id': 'eebbqej'},
 43410)

In [ ]:
print(f'numero de emoçoes: {len(dataset["train"].features["labels"].feature.names)}\n')
print(f'emoções: {dataset["train"].features["labels"].feature.names}')

numero de emoçoes: 28

emoções: ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
#instaciando o tokenizer do modelo e criando uma função para tokenizar os dados em batches
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

def tokenizer_batch(batch):
  return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

In [ ]:
#tokenizando os textos dos três conjuntos
dataset['train'] = dataset['train'].map(tokenizer_batch, batched=True)
dataset['validation'] = dataset['validation'].map(tokenizer_batch, batched=True)
dataset['test'] = dataset['test'].map(tokenizer_batch, batched=True)

In [ ]:
#verificando as colunas que o tokenizer criou
dataset['train'].column_names

['text', 'labels', 'id', 'input_ids', 'attention_mask']

In [ ]:
#
def formatar_labels(batch):
    num_labels = 28
    novas_labels = []

    for labels in batch["labels"]:
        vetor = torch.zeros(num_labels)
        for label in labels:
            vetor[label] = 1
        novas_labels.append(vetor)

    batch["labels"] = novas_labels
    return batch

dataset = dataset.map(formatar_labels, batched=True)

In [ ]:
#passando os dados do dataset para tensores
colunas_modelo = ["input_ids", "attention_mask", "labels"]

dataset['train'].set_format("torch", columns=colunas_modelo)
dataset['validation'].set_format("torch", columns=colunas_modelo)
dataset['test'].set_format("torch", columns=colunas_modelo)

In [ ]:
#defindo os batches
batch_treino = DataLoader(dataset["train"], batch_size=16, shuffle=True)
batch_validacao = DataLoader(dataset["validation"], batch_size=16)

In [ ]:
#treinamento do modelo
modelo = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=28, problem_type="multi_label_classification")

In [ ]:
#passando o modelo para rodar na GPU, pois vamos usar uma GPU T4 para o fine-tuning
gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo.to(gpu)

In [ ]:
#instaciando o otimizador
otimizador = AdamW(modelo.parameters(), lr=2e-5)

In [ ]:
#treinamento do modelo: 4 epocas com batch size de 16, ou seja, a cada 16 amostras o modelo atualiza os pesos.
epocas = 4

for epoca in range(epocas):
    modelo.train()
    total_loss = 0

    for batch in batch_treino:
        ids = batch["input_ids"].to(gpu)
        mask = batch["attention_mask"].to(gpu)
        labels = batch["labels"].to(gpu).float()

        output = modelo(input_ids=ids, attention_mask=mask, labels=labels)
        loss = output.loss

        total_loss += loss.item()

        loss.backward()
        otimizador.step()
        otimizador.zero_grad()

    print(f"Época {epoca+1}/{epocas} | Perda média: {total_loss / len(batch_treino):.4f}")

Época 1/4 | Perda média: 0.1193
Época 2/4 | Perda média: 0.0851
Época 3/4 | Perda média: 0.0758
Época 4/4 | Perda média: 0.0676
